<a href="https://colab.research.google.com/github/mbatley/apachebeam/blob/main/composite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 73.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 33.8 M

In [5]:
import apache_beam as beam
p1 = beam.Pipeline()

class MyTransform(beam.PTransform):
  def expand(self, input_col):
    a = (
        input_col
                   | 'Calculate Sum' >> beam.Map(CalculateSum)
                   | 'Apply Formatting' >> beam.Map(FormatText)
    )
    return a

def SplitRow(input_element):
  return input_element.split(',')

def FilterBasedonCountry(countryName,input_element):
  return input_element[1]==countryName

def CalculateSum(elem):
  return elem[0],(int(elem[2])+int(elem[3])+int(elem[4]))

def FormatText(elem):
  return elem[0]+' has received '+str(elem[1]) +' marks'

input_collection = (
    p1
    |beam.io.ReadFromText('students_marks.txt')
    |beam.Map(SplitRow)
  )

US_pipeline = (
   input_collection
   | beam.Filter(lambda record: FilterBasedonCountry('US',record))
   | "Composite Transformation for US" >> MyTransform()
   | 'Writing results to US File' >> beam.io.WriteToText('US_Result')
)

India_pipeline = (
   input_collection
   | beam.Filter(lambda record: FilterBasedonCountry('IN',record))
   | "Composite Transformation for IN" >> MyTransform()
   |'Writing results to India File' >> beam.io.WriteToText('IN_Result')
)


p1.run()

In [6]:
! cat US_Result-00000-of-00001

John has received 82 marks
Sam has received 89 marks


In [7]:
! cat IN_Result-00000-of-00001

Jim has received 100 marks
Sandhya has received 118 marks
Naina has received 126 marks
